In [ ]:
import os
import ansys.speos.script as script
from ansys.speos.core import Speos


tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

Create connection with Speos rpc server

In [ ]:
speos = Speos(host="localhost", port=50098)

Load Speos model.
This model contains an intentional Volume conflict to create error rays

In [ ]:
p = script.Project(speos=speos, path=os.path.join(tests_data_path, "error_data.speos", "error_data.speos"))

preview model

In [ ]:
p.preview()

In [ ]:
sim=p.find("Direct.1")[0]
sim.compute_CPU()

If we have a look to the simulation report we will find that we have 40% simulation error

In [ ]:
import ansys.speos.workflow.open_result as ORF
from IPython.display import HTML, display
data = ORF._find_correct_result(sim,"html")
display(HTML(data, metadata=dict(isolated=True)))


we will define an interactive simulation to have a look at the rays in error

In [ ]:
interactive_sim=p.create_simulation("error", feature_type=script.simulation.Interactive)
interactive_sim.set_light_expert(True)
interactive_sim.set_sensor_paths(["Irradiance.1:70"])
interactive_sim.set_source_paths(["Surface.1:4830"])
interactive_sim.commit()
results = interactive_sim.compute_CPU()
path = ORF._find_correct_result(interactive_sim, "lpf")
lxp = script.LightPathFinder(speos,path)
lxp.preview(project=p)

We can directly see a volume conflict between two solids lets filter the rays to see only rays in error

In [ ]:
lxp.filter_error_rays()
lxp.preview(project=p, ray_filter=True)